In [536]:
import pandas as pd

In [537]:
# Input the dataset 
input = 'Cocktails Dataset.xlsx'
excel_sheets = pd.ExcelFile(input).sheet_names
excel_sheets

['Cocktails', 'Sourcing', 'Conversion Rates']

In [538]:
cocktails = pd.read_excel(input, sheet_name='Cocktails')
cocktails

,Cocktail,Price (£),Recipe (ml)
0,Raspberry Lemon Drop,8.50,Citroen Vodka:45ml; Chambord:20ml; Triple Sec:...
1,Bay Breeze,7.20,Plain Vodka:60ml; Cranberry Juice:90ml; Pineap...
2,Alabama Slammer,8.25,Southern Comfort:15ml; Sloe Gin:15ml; Amaretto...
3,Watermelon Man,7.00,Plain Vodka:60ml; Watermelon Schapps:30ml; Coi...
4,Orange Blossom,8.70,London Dry Gin:30ml; Cointreau:10ml; Orange Ju...


In [539]:
sourcing = pd.read_excel(input, sheet_name='Sourcing')
sourcing

,Ingredient,Price,ml per Bottle,Currency
0,Citroen Vodka,19.25,500.0,Euro
1,Chambord,22.85,450.0,Euro
2,Triple Sec,12.00,400.0,Dollar
3,Plain Vodka,15.24,500.0,Euro
4,Cranberry Juice,1.33,1000.0,Pound
5,Pineapple Juice,1.80,1000.0,Pound
6,Southern Comfort,20.99,750.0,Dollar
7,Sloe Gin,22.99,500.0,Euro
8,Amaretto,16.60,500.0,Euro
9,Orange Juice,1.42,1000.0,Pound


In [540]:
conversion_rates = pd.read_excel(input, sheet_name='Conversion Rates')
conversion_rates

,Currency,Conversion Rate £
0,Euro,1.14
1,Dollar,1.38
2,Pound,1.00


In [541]:
# Split out the recipes into the different ingredients and their measurements
cocktails[['0','1','2','3']] = cocktails['Recipe (ml)'].str.split('; ', expand=True)
cocktails.drop(columns='Recipe (ml)', inplace=True)
cocktails = cocktails.melt(id_vars=['Cocktail', 'Price (£)'], value_vars=['0','1','2','3'], var_name='pivot_name', value_name='pivot_value')
cocktails.drop(columns='pivot_name', inplace=True)
cocktails[['Ingredient','Measure']] = cocktails['pivot_value'].str.split(':', expand=True)
cocktails.drop(columns='pivot_value', inplace=True)
cocktails.dropna(subset=['Ingredient', 'Measure'], inplace=True)
cocktails['Measure'] = cocktails['Measure'].str.replace('ml','').astype(int)
cocktails = cocktails.reset_index().drop(columns='index')

cocktails

,Cocktail,Price (£),Ingredient,Measure
0,Raspberry Lemon Drop,8.50,Citroen Vodka,45
1,Bay Breeze,7.20,Plain Vodka,60
2,Alabama Slammer,8.25,Southern Comfort,15
3,Watermelon Man,7.00,Plain Vodka,60
4,Orange Blossom,8.70,London Dry Gin,30
5,Raspberry Lemon Drop,8.50,Chambord,20
6,Bay Breeze,7.20,Cranberry Juice,90
7,Alabama Slammer,8.25,Sloe Gin,15
8,Watermelon Man,7.00,Watermelon Schapps,30
9,Orange Blossom,8.70,Cointreau,10


In [542]:
# Calculate the price in pounds, for the required measurement of each ingredient

sourcing = pd.merge(left=sourcing, right=conversion_rates, how='inner', on='Currency')
sourcing['Price'] = sourcing['Price'] / sourcing['Conversion Rate £']
sourcing['Price'] = sourcing['Price'].round(2)
sourcing

,Ingredient,Price,ml per Bottle,Currency,Conversion Rate £
0,Citroen Vodka,16.89,500.0,Euro,1.14
1,Chambord,20.04,450.0,Euro,1.14
2,Plain Vodka,13.37,500.0,Euro,1.14
3,Sloe Gin,20.17,500.0,Euro,1.14
4,Amaretto,14.56,500.0,Euro,1.14
5,Watermelon Schapps,17.36,500.0,Euro,1.14
6,Triple Sec,8.70,400.0,Dollar,1.38
7,Southern Comfort,15.21,750.0,Dollar,1.38
8,Cointreau,11.30,500.0,Dollar,1.38
9,Cranberry Juice,1.33,1000.0,Pound,1.00


In [543]:
# Join the ingredient costs to their relative cocktails
# Find the total cost of each cocktail 

output = pd.merge(left=cocktails, right=sourcing, how='inner', on='Ingredient')
# output.info()
output['Cost'] = output['Price'] / output['ml per Bottle']
output['Cost'] = output['Cost'] * output['Measure']
output['Cost'] = output['Cost'].round(2)
output = output.groupby('Cocktail').agg(Cost=('Cost','sum'), Price=('Price (£)','mean')).reset_index()

output

,Cocktail,Cost,Price
0,Alabama Slammer,1.52,8.25
1,Bay Breeze,1.77,7.20
2,Orange Blossom,0.88,8.70
3,Raspberry Lemon Drop,2.84,8.50
4,Watermelon Man,3.58,7.00


In [544]:
# Include a calculated field for the profit margin i.e. the difference between each cocktail's price and it's overall cost 
output['Margin'] = output['Price'] - output['Cost']

output

,Cocktail,Cost,Price,Margin
0,Alabama Slammer,1.52,8.25,6.73
1,Bay Breeze,1.77,7.20,5.43
2,Orange Blossom,0.88,8.70,7.82
3,Raspberry Lemon Drop,2.84,8.50,5.66
4,Watermelon Man,3.58,7.00,3.42
